# Identification Process
1. A new test muzzle image is passed through the same Gabor+Blob pipeline.

2. Extracted features are compared against the feature CSV using Cosine Similarity:
$$
\cos(\theta) = \frac{A \cdot B}{\|A\|\|B\|}
$$

3. The cattle ID with the highest similarity score is selected as the predicted identity.


In [6]:
import cv2
import numpy as np
import pandas as pd
from skimage.filters import gabor
from sklearn.metrics.pairwise import cosine_similarity
import os

def extract_muzzle_features(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"Image not found: {image_path}")

    # Gabor filter
    frequency = 0.2
    gabor_res, _ = gabor(img, frequency=frequency)
    gabor_img = (gabor_res * 255).astype(np.uint8)

    # Blob detection
    params = cv2.SimpleBlobDetector_Params()
    params.filterByArea = True
    params.minArea = 5
    params.maxArea = 300
    params.filterByCircularity = False
    params.filterByConvexity = False
    params.filterByInertia = False
    detector = cv2.SimpleBlobDetector_create(params)
    keypoints = detector.detect(gabor_img)

    # Feature vector
    blob_count = len(keypoints)
    gabor_mean = np.mean(gabor_img)
    gabor_std = np.std(gabor_img)

    return [blob_count, gabor_mean, gabor_std]

def match_to_dataset(new_feature, csv_path):
    df = pd.read_csv(csv_path, skiprows=1, names=["cattle_id", "image_name", "blob_count", "gabor_mean", "gabor_std"])


    features = df[["blob_count", "gabor_mean", "gabor_std"]].values.astype(float)
    cattle_ids = df["cattle_id"].values

    # Cosine similarity
    sims = cosine_similarity([new_feature], features)[0]
    best_idx = np.argmax(sims)
    return cattle_ids[best_idx], sims[best_idx]

# === Run Matching ===
if __name__ == "__main__":
    test_image = r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\Muzzle-Based-Biometric-Identification-System-For-Cattles-in-Nepal\resized_dataset\cattle-390\IMG20230924145902_01.jpg"
    csv_path = r"C:\Users\Amrit Shah\Desktop\Muzzle based Identification\muzzle_features.csv"

    feature = extract_muzzle_features(test_image)
    match_id, score = match_to_dataset(feature, csv_path)

    print(f"✅ Matched Cattle ID: {match_id} | Similarity Score: {score:.4f}")


✅ Matched Cattle ID: cattle-337 | Similarity Score: 1.0000
